### We are using the same steps as before with different data and see if it improves our results. We got better results here with categorical accuracy ranging from 92.66% to 95.67% which is a good amount of improvement.


In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
from sklearn.model_selection import train_test_split
import re
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import class_weight

tf.get_logger().setLevel('ERROR')


In [230]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42



In [231]:
text=pd.read_csv(r'C:\Users\Dell\Documents\projects\hate speech detection\hate-speech-and-offensive-language-master\data2\HateSpeechDetection.csv')

In [232]:
text=text.drop(columns=['Platform'])

In [233]:
text['Hateful'].value_counts()

0    2400
1     600
Name: Hateful, dtype: int64

In [234]:
text['Comment']=text['Comment'].fillna('').astype(str)

In [235]:
x=text['Comment'].copy()
y=text['Hateful'].copy()

In [236]:
x

0       Damn I thought they had strict gun laws in Ger...
1       I dont care about what it stands for or anythi...
2                       It's not a group it's an idea lol
3                               So it's not just America!
4       The dog is a spectacular dancer considering he...
                              ...                        
2995    This is what Kike shilling looks like. Ryan ha...
2996                              Not bait, they're right
2997                               I like this one a lot.
2998             Kikes making money off heroin what’s new
2999    Desecrate men by making them gay/trannies, and...
Name: Comment, Length: 3000, dtype: object

In [237]:
x=pd.DataFrame({'ind':x.index, 'Comment':x.values})


In [238]:
x

,ind,Comment
0,0,Damn I thought they had strict gun laws in Ger...
1,1,I dont care about what it stands for or anythi...
2,2,It's not a group it's an idea lol
3,3,So it's not just America!
4,4,The dog is a spectacular dancer considering he...
...,...,...
2995,2995,This is what Kike shilling looks like. Ryan ha...
2996,2996,"Not bait, they're right"
2997,2997,I like this one a lot.
2998,2998,Kikes making money off heroin what’s new


In [239]:
x=x.drop(columns=['ind'])

In [240]:
y=pd.DataFrame(y)

In [241]:
y

,Hateful
0,0
1,0
2,0
3,0
4,0
...,...
2995,1
2996,0
2997,0
2998,1


In [242]:
xtrain1,xtest1,ytrain1,ytest1=train_test_split (x['Comment'], y['Hateful'] , test_size=0.2)
xtest1,xval1,ytest1,yval1=train_test_split (xtest1, ytest1 , test_size=0.5)

In [243]:
xtrain1=pd.DataFrame(xtrain1)
xtest1=pd.DataFrame(xtest1)
xval1=pd.DataFrame(xval1)


In [244]:
xtrain1['class']=ytrain1.copy()
xtest1['class']=ytest1.copy()
xval1['class']=yval1.copy()

In [245]:
xval1

,Comment,class
1423,"The vast world, want to meet a bosom, really h...",0
1342,"Damn, two very different characters having a r...",0
2391,I remember a couple years back some negres ape...,1
472,Who the fuck is filming and why?,0
410,Largest pedophile organization in the world. ...,1
...,...,...
1421,Liar. Find out what happened to all the covid...,0
55,I find this very annoying.,0
1985,You're the retards forcing it onto other peopl...,1
2844,Those dramatic Jewish Nazi larper faggots defi...,1


In [246]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [247]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [248]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_word_ids', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [249]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.76262873  0.9928099  -0.18611842  0.36673835  0.15233713  0.6550445
  0.9681154  -0.94862705  0.00216199 -0.9877732   0.06842697 -0.97630596]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.28946307  0.3432126   0.33231524 ...  0.21300834  0.71020776
  -0.05771159]
 [-0.28742072  0.31980997 -0.23018596 ...  0.5845511  -0.21329741
   0.7269215 ]
 [-0.66157013  0.6887673  -0.8743302  ...  0.10877225 -0.26173237
   0.47855318]
 ...
 [-0.22561178 -0.2892561  -0.07064433 ...  0.4756602   0.83277094
   0.4002539 ]
 [-0.29824233 -0.27473113 -0.05450515 ...  0.48849773  1.0955355
   0.18163365]
 [-0.4437818   0.00930784  0.07223748 ...  0.17290097  1.183325
   0.07897963]]


In [250]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  #net = tf.keras.layers.Dense(100, activation='relu')(net)
  net = tf.keras.layers.Dense(2, activation='softmax', name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [251]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(['hello ']))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.6572732 0.5863348]], shape=(1, 2), dtype=float32)


In [143]:
tf.keras.utils.plot_model(classifier_model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [252]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
metrics = tf.metrics.BinaryAccuracy()

In [253]:
epochs = 10
steps_per_epoch = len(xtrain1)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [254]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [255]:
xtrain1

,Comment,class
2059,The white race keeps catching those L's.,1
1535,What is the female equivalent of that?,0
2125,he apologized to a heckler. no integrity as a ...,0
694,When are we gonna address the PPP scams that w...,0
1627,So I'm talking to my friend and this nigger wo...,1
...,...,...
2640,"best in thread, rest are boring or poorly comp...",0
2438,Not sure about rent but I'd take Ireland over ...,0
790,Scumbag thing to do posting animal cruelty vid...,1
946,"Fuck u and all ur friends ugly motherfucker, g...",1


In [256]:
xtrain1=xtrain1.drop(columns=['class'])
xtest1=xtest1.drop(columns=['class'])
xval1=xval1.drop(columns=['class'])

In [257]:
ytrain1=pd.get_dummies(ytrain1)
yval1=pd.get_dummies(yval1)
ytest1=pd.get_dummies(ytest1)

In [258]:
ytrain1.columns=['normal','hatespeech']
yval1.columns=['normal','hatespeech']
ytest1.columns=['normal','hatespeech']

In [271]:
ytrain1

,normal,hatespeech
2059,0,1
1535,1,0
2125,1,0
694,1,0
1627,0,1
...,...,...
2640,1,0
2438,1,0
790,0,1
946,0,1


In [260]:
class_weights = {0:4,1:1}

In [261]:
print(f'Training model with {tfhub_handle_encoder}')
new_mod = classifier_model.fit(xtrain1,ytrain1, validation_data= (xval1,yval1), 
                               epochs=epochs, class_weight=class_weights)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/10
75/75 [==============================] - 441s 6s/step - loss: 1.4596 - binary_accuracy: 0.7950 - val_loss: 0.5045 - val_binary_accuracy: 0.8300
Epoch 2/10
75/75 [==============================] - 461s 6s/step - loss: 0.9393 - binary_accuracy: 0.8004 - val_loss: 0.5170 - val_binary_accuracy: 0.8300
Epoch 3/10
75/75 [==============================] - 457s 6s/step - loss: 0.7524 - binary_accuracy: 0.8004 - val_loss: 0.5075 - val_binary_accuracy: 0.8300
Epoch 4/10
75/75 [==============================] - 458s 6s/step - loss: 0.6763 - binary_accuracy: 0.8004 - val_loss: 0.4412 - val_binary_accuracy: 0.8300
Epoch 5/10
75/75 [==============================] - 456s 6s/step - loss: 0.5598 - binary_accuracy: 0.8025 - val_loss: 0.3171 - val_binary_accuracy: 0.8500
Epoch 6/10
75/75 [==============================] - 457s 6s/step - loss: 0.4088 - binary_accuracy: 0.8642 - val_loss: 0.2184 - val_bi

In [190]:
vals=classifier_model.predict(xtest1)

10/10 [==============================] - 11s 1s/step


In [191]:
classifier_model.evaluate(xtest1,ytest1)

10/10 [==============================] - 11s 1s/step - loss: 0.1965 - binary_accuracy: 0.9267


[0.19650669395923615, 0.9266666769981384]

In [272]:
ytest1

,normal,hatespeech
464,1,0
2132,1,0
1710,1,0
2735,1,0
2919,1,0
...,...,...
516,1,0
1399,1,0
1908,0,1
434,0,1


In [192]:
vals=np.rint(vals)
vals

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.

In [199]:
count=0
for ele in temp[0]:
    if ele==1:
        count+=1

In [200]:
count

227

In [193]:
temp=pd.DataFrame(vals)

In [195]:
temp

,0,1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0
...,...,...
295,1.0,0.0
296,1.0,0.0
297,1.0,0.0
298,1.0,0.0


In [290]:
classifier_model.save("C:\\Users\\Dell\\Documents\\projects\\hate speech detection\\hate-speech-and-offensive-language-master\\embeddings\\bert2_data2_2nd")

In [289]:
classifier_model.predict(["""he is an cute lil idiot"""])

1/1 [==============================] - 0s 135ms/step


array([[0.96850103, 0.03149895]], dtype=float32)

# load the saved model

In [291]:
loaded=tf.keras.models.load_model("C:\\Users\\Dell\\Documents\\projects\\hate speech detection\\hate-speech-and-offensive-language-master\\embeddings\\bert2_data2_2nd",compile=False)

In [436]:
loaded.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [298]:
loaded.predict(["""indians"""])

1/1 [==============================] - 0s 160ms/step


array([[0.07990883, 0.92009115]], dtype=float32)